In [4]:
import torch
from torch import nn
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
import re
from ipywidgets import interact_manual
import unicodedata
import sys
 
# setting path
sys.path.append('../')
from modules.encoder import Encoder
from modules.utils import create_sequence_mask

In [8]:
train_data = pd.read_csv('train_pr.csv')
val_data = pd.read_csv('val_pr.csv')

In [9]:
def preprocess_text(text:str):
    text = unicodedata.normalize('NFC',text)
    text = re.sub(r'&[a-z]+?;','',text)
    text = re.sub(r'<[^>]+>',' _html_tag_ ',text)
    text = re.sub(r'[a-zA-Z0-9-_][.a-zA-Z0-9-_]*@([.a-zA-Z0-9_-]+){,3}',' _email_ ',text)
    text = re.sub(r'https?://[a-zA-Z0-9-_&.?/%=]*',' _url_link_ ',text)
    text = re.sub(r'#[_a-zA-Z0-9]+',' _hash_tag_ ',text)
    text = re.sub(r'@[_a-zA-Z0-9]+',' _tag_ ',text)
    text = re.sub(r'[0-9][0-9.,]*',' _num_ ',text)
    text = re.sub(r'[^\sa-zA-Z0-9.,!;:/\'^()=<>_?]','',text)
    text = re.sub(r'(\s*[.!?:;-]\s*)+',r' \1 ', text)
    
    return ' '.join(word_tokenize(text)).lower()

In [26]:
@interact_manual
def _(text=''):
    return preprocess_text(text)

interactive(children=(Text(value='', description='text'), Button(description='Run Interact', style=ButtonStyle…

In [22]:
"""
tqdm.pandas()
train_data['SentimentText'] = train_data['SentimentText'].progress_apply(preprocess_text)
train_data.to_csv('train_pr.csv',index=False)
val_data['SentimentText'] = val_data['SentimentText'].progress_apply(preprocess_text)
val_data.to_csv('val_pr.csv',index=False)
"""

In [10]:
from collections import Counter

PAD = "[PAD]"
UNK = "[UNK]"


def build_vocab(df, min_freq=3):
    counter = Counter()
    for tweet in df.itertuples():
        counter.update(tweet.SentimentText.split())

    vocab = sorted([w for w, c in counter.items() if c >= min_freq])

    w2i = {w: i for i, w in enumerate([PAD, UNK] + vocab)}
    i2w = {v: k for k, v in w2i.items()}

    return w2i, i2w


w2i, i2w = build_vocab(train_data)

In [11]:
from torch.utils.data import DataLoader, Dataset


class TweetDataset(Dataset):
    def __init__(self, df, vocab):
        self.df = df
        self.vocab = vocab

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index: int):
        text = self.df["SentimentText"].values[index].split()

        return (
            self.df["Sentiment"].values[index],
            [self.vocab.get(w, self.vocab[UNK]) for w in text],
            len(text),
        )


class TweetDatasetCollate:
    def __init__(self, max_length, vocab):
        self.max_length = max_length
        self.vocab = vocab

    def __call__(self, batch):
        labels = []
        ids = []
        lens = []

        for label, idx, l in batch:
            labels.append(label)
            ids.append(
                idx[: self.max_length]
                + max(0, self.max_length - len(idx)) * [self.vocab[PAD]]
            )
            lens.append(l if l <= self.max_length else self.max_length)

        return torch.LongTensor(labels), torch.LongTensor(ids), torch.LongTensor(lens)

In [12]:
class EncoderClassifier(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        n_heads: int,
        max_length: int,
        n_blocks: int,
        d_model: int,
        d_ff: int,
        d_k: int,
        d_v: int,
        n_classes: int,
        p_drop: float,
    ):
        super().__init__()
        self.max_length = max_length
        self.encoder = Encoder(
            vocab_size, n_heads, max_length, n_blocks, d_model, d_ff, d_k, d_v, p_drop
        )
        self.fc = nn.Linear(d_model, n_classes)
        self.dropout = nn.Dropout(p=p_drop)

    def forward(self, x: torch.Tensor, x_lengths: torch.Tensor):
        x_mask = create_sequence_mask(x_lengths, self.max_length)
        x = torch.relu(self.encoder(x, x_mask))
        return self.fc(self.dropout((x * x_mask.squeeze().unsqueeze(-1)).mean(dim=1)))

In [13]:
vocab_size = len(w2i)
max_length = 128
n_heads = 8
d_model = 128
n_blocks = 4
d_ff = 128
d_k = d_v = d_model // n_heads
p_drop = 0.4
n_classes = 2
batch_size = 128

In [14]:
collate = TweetDatasetCollate(max_length, w2i)
train_dl = DataLoader(
    TweetDataset(train_data, w2i),
    shuffle=True,
    batch_size=batch_size,
    collate_fn=collate,
)
val_dl = DataLoader(
    TweetDataset(val_data, w2i), batch_size=batch_size * 2, collate_fn=collate
)

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu');device

device(type='cuda')

In [104]:
clf = EncoderClassifier(
    vocab_size,
    n_heads,
    max_length,
    n_blocks,
    d_model,
    d_ff,
    d_k,
    d_v,
    n_classes,
    p_drop,
).to(device)

for p in clf.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

In [ ]:
from torch.optim import Adam
from torch.optim.lr_scheduler import ExponentialLR

optimizer = Adam(clf.parameters(), lr=0.0008, betas=(0.9, 0.98), weight_decay=0.001)
loss_fn = nn.CrossEntropyLoss()
scheduler = ExponentialLR(optimizer, 0.999**0.125)

In [106]:
import os

os.makedirs("checkpoints", exist_ok=True)

def train_and_eval(
    model,
    optimizer,
    loss_fn,
    scheduler,
    epochs,
    train_dl,
    val_dl,
    early_stopping=5,
    model_name="sample_model",
):
    p_bar = tqdm(total=len(train_dl))

    best_val_loss = 25042001
    patience = 0

    for epoch in range(epochs):
        train_loss = 0
        train_corr = 0
        val_loss = 0
        val_corr = 0

        model.train()
        for y, x, x_len in train_dl:
            y, x, x_len = y.to(device), x.to(device), x_len.to(device)

            logits = model(x, x_len)
            loss = loss_fn(logits, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_corr += (y == logits.argmax(-1)).sum().item()
            p_bar.update(1)
        scheduler.step()

        model.eval()
        with torch.inference_mode():
            for y, x, x_len in val_dl:
                y, x, x_len = y.to(device), x.to(device), x_len.to(device)

                logits = model(x, x_len)

                val_loss += loss_fn(logits, y).item()
                val_corr += (y == logits.argmax(-1)).sum().item()

        train_loss /= len(train_dl)
        val_loss /= len(val_dl)
        train_acc = train_corr / len(train_dl.dataset)
        val_acc = val_corr / len(val_dl.dataset)

        if val_loss > best_val_loss:
            patience += 1
        else:
            torch.save(
                model.state_dict(),
                f"checkpoints/{model_name}_{(epoch+1)*len(train_dl)}.pth",
            )
            patience = 0
            best_val_loss = val_loss

        print(
            f"Epoch {epoch+1}:\n Train loss: {train_loss:.6f} - Train acc: {train_acc:.6f}\n Val loss: {val_loss:.6f} - Val acc: {val_acc:.6f}"
        )

        if patience >= early_stopping:
            print(
                f"Stopped since val loss has not improved in the last {early_stopping} epochs..."
            )
            break

        p_bar.reset()

In [107]:
epochs = 100
train_and_eval(clf, optimizer, loss_fn, scheduler, epochs, train_dl, val_dl)

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch 1:
 Train loss: 0.561814 - Train acc: 0.723506
 Val loss: 0.506942 - Val acc: 0.766727
Epoch 2:
 Train loss: 0.508019 - Train acc: 0.766624
 Val loss: 0.492749 - Val acc: 0.769327
Epoch 3:
 Train loss: 0.498705 - Train acc: 0.769599
 Val loss: 0.492737 - Val acc: 0.767727
Epoch 4:
 Train loss: 0.494587 - Train acc: 0.770887
 Val loss: 0.500492 - Val acc: 0.761826
Epoch 5:
 Train loss: 0.492521 - Train acc: 0.772349
 Val loss: 0.483920 - Val acc: 0.770077
Epoch 6:
 Train loss: 0.490277 - Train acc: 0.774775
 Val loss: 0.492605 - Val acc: 0.764826
Epoch 7:
 Train loss: 0.488810 - Train acc: 0.774550
 Val loss: 0.483440 - Val acc: 0.772477
Epoch 8:
 Train loss: 0.488370 - Train acc: 0.775487
 Val loss: 0.495463 - Val acc: 0.759676
Epoch 9:
 Train loss: 0.486715 - Train acc: 0.773912
 Val loss: 0.490272 - Val acc: 0.765027
Epoch 10:
 Train loss: 0.486704 - Train acc: 0.775125
 Val loss: 0.482875 - Val acc: 0.768127
Epoch 11:
 Train loss: 0.485783 - Train acc: 0.777225
 Val loss: 0.49

In [29]:
from sklearn.metrics import f1_score

pretrained = EncoderClassifier(
    vocab_size,
    n_heads,
    max_length,
    n_blocks,
    d_model,
    d_ff,
    d_k,
    d_v,
    n_classes,
    p_drop,
)

pretrained.load_state_dict(torch.load("checkpoints/sample_model_13125.pth"))
pretrained.to(device)
pretrained.eval()

y_true = []
y_pred = []
val_corr = 0
with torch.inference_mode():
    for y, x, x_len in val_dl:
        y, x, x_len = y.to(device), x.to(device), x_len.to(device)

        y_hat = pretrained(x, x_len).argmax(-1)
        val_corr += (y == y_hat).sum().item()
        y_true += y.tolist()
        y_pred += y_hat.tolist()

print(f"Best acc: {val_corr/len(val_dl.dataset):.6f}")
print(f"Best f1: {f1_score(y_true,y_pred):.6f}")

Best acc: 0.775578
Best f1: 0.802395
